<a href="https://colab.research.google.com/github/erichsiao1106/cnn/blob/master/lymphoma_generator_tf_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
from glob import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.models import Model

In [ ]:
IMG_SIZE = 200
BATCH_SIZE = 16
class_map = {'CLL':0, 'MCL': 1, 'FL': 2}

In [ ]:
# Read single image
data_path = 'drive/My Drive/class/緯育醫學影像/Day4/lymphoma/*/*.tif' 
img_paths = glob(data_path)
img_paths_train, img_paths_test = train_test_split(img_paths, test_size=0.2)
len(img_paths), len(img_paths_train), len(img_paths_test)

(374, 299, 75)

In [ ]:
def data_generator(data_paths, batch_size, num_classes):
    '''data generator for fit'''
    n = len(data_paths)
    i = 0
    data_paths = data_paths
    while True:
        image_data = [] # X: model input
        class_data = [] # y: label
        for b in range(batch_size):
            if i==0:
                np.random.shuffle(data_paths)
            path = data_paths[i]
            img = cv2.imread(path)
            img_resize = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img_resize = img_resize/255

            cls = path.split('/')[-2]

            image_data.append(img_resize)
            class_data.append(class_map[cls])
            i = (i+1) % n
        image_data = np.array(image_data)
        class_data = np.array(class_data)
        class_data = tf.keras.utils.to_categorical(class_data, num_classes=num_classes)
        yield image_data, class_data


In [ ]:
generator_train = data_generator(data_paths=img_paths_train, batch_size=BATCH_SIZE, num_classes=3)
generator_test = data_generator(data_paths=img_paths_test, batch_size=BATCH_SIZE, num_classes=3)

In [ ]:
# 1
inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = Conv2D(filters=32, kernel_size=3, activation='relu')(inputs)
x = Conv2D(filters=32, kernel_size=3, activation='relu')(x)
x = MaxPool2D(2)(x)
x = Conv2D(filters=64, kernel_size=3, activation='relu')(x)
x = Conv2D(filters=64, kernel_size=3, activation='relu')(x)
x = MaxPool2D(2)(x)
x = Conv2D(filters=128, kernel_size=3, activation='relu')(x)
x = Conv2D(filters=128, kernel_size=3, activation='relu')(x)
x = MaxPool2D(2)(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(64, activation=tf.keras.activations.relu)(x)
prediction = Dense(3, activation='softmax')(x)
model = Model(inputs=inputs, outputs=prediction)

In [ ]:
model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 200, 200, 3)]     0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 198, 198, 32)      896       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 196, 196, 32)      9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 98, 98, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 96, 96, 64)        18496     
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 94, 94, 64)        36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 47, 47, 64)       

In [ ]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
logs = model.fit(generator_train,
                epochs=10,
                steps_per_epoch=len(img_paths_train)//BATCH_SIZE,
                validation_data=generator_test,
                validation_steps=len(img_paths_test)//BATCH_SIZE,
                )

Epoch 1/10
18/18 [==============================] - 8s 421ms/step - loss: 1.1009 - accuracy: 0.3646 - val_loss: 1.0982 - val_accuracy: 0.3750
Epoch 2/10
18/18 [==============================] - 8s 420ms/step - loss: 1.0939 - accuracy: 0.3750 - val_loss: 1.0857 - val_accuracy: 0.3750
Epoch 3/10
18/18 [==============================] - 8s 423ms/step - loss: 1.0657 - accuracy: 0.4375 - val_loss: 1.0370 - val_accuracy: 0.5000
Epoch 4/10
18/18 [==============================] - 8s 424ms/step - loss: 0.9730 - accuracy: 0.5208 - val_loss: 0.9586 - val_accuracy: 0.5469
Epoch 5/10
18/18 [==============================] - 8s 429ms/step - loss: 0.9580 - accuracy: 0.5417 - val_loss: 0.9987 - val_accuracy: 0.4688
Epoch 6/10
18/18 [==============================] - 8s 423ms/step - loss: 0.9864 - accuracy: 0.4965 - val_loss: 0.9915 - val_accuracy: 0.4688
Epoch 7/10
18/18 [==============================] - 8s 424ms/step - loss: 0.9193 - accuracy: 0.5694 - val_loss: 0.9666 - val_accuracy: 0.5000
Epoch 

In [ ]:
history = logs.history

In [ ]:
plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.legend(['accuracy', 'val_accuracy'])
plt.title('accuracy')

In [ ]:
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.title('loss')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_true = np.argmax(y_test, axis=-1)
y_pred = np.argmax(model.predict(X_test), axis=-1)
print(y_true.shape, y_pred.shape)
target_names = [str(i) for i in range(3)]
print(classification_report(y_true, y_pred, target_names=target_names))
print(confusion_matrix(y_true, y_pred))
